# Real Time ASL Alphabet Recognition system


## Checking the Structure of the dataset

In [11]:
import os

def print_dataset_structure(root_dir, indent=""):
    for item in sorted(os.listdir(root_dir)):
        item_path = os.path.join(root_dir, item)
        if os.path.isdir(item_path):
            print(f"{indent}[DIR] {item}")
            print_dataset_structure(item_path, indent + "    ")
        else:
            files = sorted(os.listdir(root_dir))[:3]
            for f in files:
                print(f"{indent}    {f}")
            break  

# Example usage:
dataset_path = "dataset"  
print_dataset_structure(dataset_path)


[DIR] asl_alphabet_test
        A_test.jpg
        B_test.jpg
        C_test.jpg
[DIR] asl_alphabet_train
    [DIR] A
            A1.jpg
            A10.jpg
            A100.jpg
    [DIR] B
            B1.jpg
            B10.jpg
            B100.jpg
    [DIR] C
            C1.jpg
            C10.jpg
            C100.jpg
    [DIR] D
            D1.jpg
            D10.jpg
            D100.jpg
    [DIR] E
            E1.jpg
            E10.jpg
            E100.jpg
    [DIR] F
            F1.jpg
            F10.jpg
            F100.jpg
    [DIR] G
            G1.jpg
            G10.jpg
            G100.jpg
    [DIR] H
            H1.jpg
            H10.jpg
            H100.jpg
    [DIR] I
            I1.jpg
            I10.jpg
            I100.jpg
    [DIR] J
            J1.jpg
            J10.jpg
            J100.jpg
    [DIR] K
            K1.jpg
            K10.jpg
            K100.jpg
    [DIR] L
            L1.jpg
            L10.jpg
            L100.jpg
    [DIR] M
            M1.jpg

## Data loading and Augmentation

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report
from tensorflow.keras.regularizers import l2

train_dir = "dataset/asl_alphabet_train"


img_height, img_width = 64, 64
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


## Model Training


In [15]:

l2_reg = 0.001

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(l2_reg), input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(l2_reg)),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(l2_reg)),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(l2_reg)),
    Dropout(0.5),
    Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │       1,179,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 29)                  │           7,453 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,605 (4.89 MB)

 Trainable params: 1,280,605 (4.89 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_gen,
    epochs=5,
    validation_data=val_gen
)


model.save("asl_cnn_model.h5")

## Plotting the Accuracy and Loss

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Loss')
plt.legend()
plt.show()